In [2]:
import numpy as np
import pandas as pd
import matplotlib. pyplot as plt
from okama.assets import Asset, AssetList, Portfolio
from okama.data import get_eod_data
from okama.helpers import Frame

pd.options.display.float_format = '{:,.4f}'.format

%load_ext autoreload
%autoreload 2

SBSP.MCX
VTBA.MCX
AKSP.MCX

In [33]:
def index_funds_eval(asset_list: list, first_date=None, last_date=None, curr='USD') -> pd.DataFrame: 
    """
    приведенная к году версия отклонения (tracking difference)
    """
    ror = AssetList(symbols=asset_list, first_date=first_date, last_date=last_date, curr=curr).ror
    m = ror.shape[0]
    print(f'количество месяцев: {m}')
    ret = (ror + 1.).prod() - 1.
    
    def tracking_difference(ret: pd.Series, nm: str) -> float:
        return ret[nm] - ret[0]
    
    def tracking_difference_annualized(ret: pd.Series, nm: str, m: float) -> float:
        x = tracking_difference(ret, nm)
        try:
            y = abs(x)
        except:
            y = np.nan
        diff = np.sign(x)*((y + 1)**(12 / m) - 1)
        return diff
    
    def tracking_error(ror: pd.DataFrame, nm: str, m: float) -> float:
        x = (((ror[nm] - ror.iloc[:,0]) ** 2).sum() / (m-1)) ** (1/2)
        x_annualized = x * np.sqrt(12)
        return x_annualized
        
    df = pd.DataFrame()
    for i, ticker in enumerate(asset_list):
        if i > 0:
            nav = Asset(ticker).nav_ts
            if nav is not np.nan:
                nav = nav[-1]
            diff = tracking_difference(ret, ticker)
            diff_a = tracking_difference_annualized(ret, ticker, m)
            error = tracking_error(ror, ticker, m)
            row = {
                'Тикер': ticker,
                'Название': Asset(ticker).name,
                'Отклонение': diff * 100,
                'Отклонение среднегодовое': diff_a *100,
                'Ошибка следования': error * 100,
                'СЧА': nav / 1_000_000
                }
            df = df.append(row, ignore_index=True)
    return df

### Фонды, следующие S&P 500

In [46]:
ls1 = ['SP500TR.INDX', 'SPY.US','SBSP.MCX', 'VTBA.MCX', 'AKSP.MCX', 'FXUS.MCX']

In [124]:
df1 = index_funds_eval(asset_list=ls1, last_date='2020-03', curr='USD')

Total time taken is 0.29 min.
количество месяцев: 9


In [125]:
df1

,Название,Отклонение,Отклонение среднегодовое,Ошибка следования,СЧА,Тикер
0,SPDR S&P 500 ETF Trust,0.1482,0.1977,0.4774,nan,SPY.US
1,SBSP,0.4339,0.5790,4.7386,nan,SBSP.MCX
2,VTBA,-0.1758,-0.2344,3.5036,nan,VTBA.MCX
3,AKSP,0.6384,0.8521,4.2018,nan,AKSP.MCX
4,FinEx Funds Public Limited Company - FinEx MSC...,0.7943,1.0605,4.3959,nan,FXUS.MCX


In [49]:
df1 = index_funds_eval(asset_list=ls1, first_date='2019-07', last_date='2020-03', curr='USD')

количество месяцев: 9
Total time taken is 0.73 min.
Total time taken is 0.02 min.
Total time taken is 0.02 min.
Total time taken is 0.02 min.
Total time taken is 0.02 min.


In [50]:
df1

,Название,Отклонение,Отклонение среднегодовое,Ошибка следования,СЧА,Тикер
0,SPDR S&P 500 ETF Trust,0.1482,0.1977,0.4774,nan,SPY.US
1,SBSP,0.4339,0.5790,4.7386,nan,SBSP.MCX
2,VTBA,-0.1758,-0.2344,3.5036,nan,VTBA.MCX
3,AKSP,0.6384,0.8521,4.2018,nan,AKSP.MCX
4,FinEx Funds Public Limited Company - FinEx MSC...,0.7943,1.0605,4.3959,nan,FXUS.MCX


In [36]:
df1.to_csv('sp500_error.csv', sep=',')

### Фонды, следующие индексу Мосбиржи

In [51]:
ls2 = ['MCFTR.INDX',
      '0295-74549871.RUFUND',
      '0662-94119196.RUFUND',
      '0695-94121916.RUFUND',
      '0278-58233720.RUFUND',
      '0177-71671092.RUFUND',
      '1149-94139555.RUFUND',
      '0890-94127385.RUFUND',
      '0983-94131725.RUFUND',
      '0351-76578546.RUFUND',
      'FXRL.MCX',
      'RUSE.MCX',
      'SBMX.MCX',
      '3555.RUFUND'
     ]

In [121]:
x = AssetList(ls2)

Total time taken is 0.86 min.


In [120]:
x = AssetList(ls2)

Total time taken is 1.15 min.


In [122]:
x.first_date

Timestamp('2018-10-01 00:00:00')

In [115]:
x.first_date

Timestamp('2018-09-01 00:00:00')

In [123]:
index_funds_eval(asset_list=ls2, first_date='2018-10', last_date='2020-03', curr='RUB')

Total time taken is 0.32 min.
количество месяцев: 18
Total time taken is 0.02 min.
Total time taken is 0.02 min.
Total time taken is 0.02 min.
Total time taken is 0.02 min.
Total time taken is 0.02 min.
Total time taken is 0.02 min.
Total time taken is 0.02 min.
Total time taken is 0.02 min.
Total time taken is 0.02 min.
Total time taken is 0.02 min.
Total time taken is 0.02 min.
Total time taken is 0.02 min.
Total time taken is 0.02 min.


,Название,Отклонение,Отклонение среднегодовое,Ошибка следования,СЧА,Тикер
0,AK BARS Indeksnyy (AK BARS KAPITAL),-6.5236,-4.3031,4.0927,18.6247,0295-74549871.RUFUND
1,Alltek - Birzhevoy indeks aktsiy (Alltek),-9.9490,-6.5273,5.6951,5.3188,0662-94119196.RUFUND
2,Alʹfa–Kapital Indeks MosBirzhi (Alʹfa-Kapital),-5.0937,-3.3676,3.0552,977.5804,0695-94121916.RUFUND
3,BKS Rossiyskiye Aktsii (Brokerkredit·servis),-4.8327,-3.1964,2.8624,527.8617,0278-58233720.RUFUND
4,VTB - Indeks MosBirzhi (VTB Kapital Upravleniy...,-2.5563,-1.6971,3.0918,"1,502.9931",0177-71671092.RUFUND
5,Ingosstrakh Indeks MosBirzhi (Ingosstrakh - In...,-7.9877,-5.2566,4.6720,43.1805,1149-94139555.RUFUND
6,Otkrytiye – Indeks MosBirzhi (OTKRYTIYe),-3.2810,-2.1755,2.7756,495.2428,0890-94127385.RUFUND
7,Rayffayzen – Indeks MMVB golubykh fishek (Rayf...,-3.8857,-2.5740,2.8883,"1,867.4334",0983-94131725.RUFUND
8,Solid - Indeks MosBirzhi (SOLID Menedzhment),-2.3377,-1.5525,5.2965,75.6402,0351-76578546.RUFUND
9,FinEx Russian RTS Equity UCITS ETF (USD),-1.7195,-1.1431,1.2810,nan,FXRL.MCX


In [39]:
ls2 = ['MCFTR.INDX',
      'SBMX.MCX',
      '3555.RUFUND'
     ]

In [41]:
get_eod_data('FXRL.MCX', type='close')

Date
2016-03-03   1,537.0000
2016-03-04   1,566.0000
2016-03-07   1,593.0000
2016-03-09   1,579.5000
2016-03-10   1,571.0000
                ...    
2020-04-06   2,661.5000
2020-04-07   2,670.5000
2020-04-08   2,688.0000
2020-04-09   2,740.0000
2020-04-10   2,729.5000
Freq: D, Name: FXRL.MCX, Length: 1036, dtype: float64

In [113]:
z = get_eod_data('BND.US', type='return')

In [114]:
z.head(20)

Date
2007-04-11   -0.0027
2007-04-12   -0.0001
2007-04-13   -0.0016
2007-04-16    0.0009
2007-04-17    0.0025
2007-04-18    0.0019
2007-04-19   -0.0004
2007-04-20   -0.0003
2007-04-23    0.0013
2007-04-24    0.0016
2007-04-25   -0.0008
2007-04-26   -0.0021
2007-04-27   -0.0002
2007-04-30    0.0033
2007-05-01   -0.0006
2007-05-02    0.0001
2007-05-03   -0.0007
2007-05-04    0.0008
2007-05-07    0.0007
2007-05-08    0.0001
Freq: D, Name: BND.US, dtype: float64

In [74]:
z.index[0]

Period('1993-01-29', 'D')

In [99]:
z.ror.index[0].to_timestamp()

AttributeError: 'Series' object has no attribute 'ror'

In [78]:
m = z.index[0].month
m

1

In [84]:
y = z.index[0].year
str(y+1)

'1994'

In [103]:
first = str(y)+'-'+str(m)

In [104]:
z[first].count()

2

### Разработка функций

### 1 год

In [2]:
x1 = AssetList(['SP500TR.INDX', 'SPY.US','SBSP.MCX'])

In [45]:
x1.first_date

Timestamp('2019-04-01 00:00:00')

In [52]:
diff1 = (x1.ror[:'2020-03'] + 1.).prod() - 1.

In [53]:
diff1

SP500TR.INDX   -0.0698
SPY.US         -0.0689
SBSP.MCX       -0.0764
dtype: float64

In [55]:
diff1['SPY.US'] - diff1['SP500TR.INDX']

0.0008728897083893905

In [57]:
(diff1['SBSP.MCX'] - diff1['SP500TR.INDX']) * 100

-0.6647852412279942

In [54]:
x1.wealth_indexes[:'2020-03']

,SP500TR.INDX,SPY.US,SBSP.MCX
Date,,,
2019-04,"1,000.0000","1,000.0000","1,000.0000"
2019-05,974.3683,974.4759,975.4826
2019-06,"1,043.0385","1,042.2852","1,030.5699"
2019-07,"1,058.0302","1,058.0439","1,061.8433"
2019-08,"1,041.2696","1,040.3287","1,031.9364"
2019-09,"1,060.7524","1,060.5701","1,049.0784"
2019-10,"1,083.7273","1,084.0138","1,071.2309"
2019-11,"1,123.0656","1,123.2530","1,113.2310"
2019-12,"1,156.9629","1,155.8895","1,138.0088"


In [40]:
x1.wealth_indexes['SPY.US'].iloc[-1] / x1.wealth_indexes['SP500TR.INDX'].iloc[-1] - 1.

0.0003016575162420043

In [41]:
x1.wealth_indexes['SBSP.MCX'].iloc[-1] / x1.wealth_indexes['SP500TR.INDX'].iloc[-1] - 1.

0.004082715207199383

In [45]:
x3 = AssetList(ls)

In [59]:
x3.first_date

Timestamp('2019-06-01 00:00:00')

In [57]:
ret = (x3.ror + 1.).prod() - 1.
ret

SP500TR.INDX    0.0164
SPY.US          0.0158
SBSP.MCX       -0.0113
VTBA.MCX       -0.0211
AKSP.MCX       -0.1411
dtype: float64

In [58]:
((x3.ror['AKSP.MCX'] - x3.ror.iloc[:,0])**2).sum()

0.022958779660885557

In [7]:
x3.ror.shape[0]

11

In [67]:
ret = ((x3.ror + 1.).prod() - 1.)*100
ret

SP500TR.INDX    -1.7348
SPY.US          -1.7160
SBSP.MCX        -1.4463
VTBA.MCX        -2.4909
AKSP.MCX       -14.5392
dtype: float64

In [73]:
ret['SP500TR.INDX']

-1.7348308743730056